# FTS band conventions

We measure our detector bands with an FTS in front of the cryostat.  These raw measurements need to be converted correctly to be used as filter response functions for integrals of on-sky sources, or to infer actual "on wafer" detector bands.

In the first case, we want to calculate the power from an on-sky source, which is given by
$$ P_{on-sky} = \int_0^\infty  B_\nu(\nu,T_{src}) \cdot A\Omega(\nu) 
    \cdot F_{on-sky}(\nu) d\nu .$$

Here we discuss how to find $F_{on-sky}(\nu)$ and $F_{detector}$ from the measured FTS spectra.  We consider three frequency-dependent corrections:

 - $F_{splitter}(\nu)$ = frequency dependence of the coupling to the FTS.  Here we assume reflection off a thin dielectric, which has an intensity reflection proportional to $\nu^2$.
 - $F_{optics}(\nu)$ = frequency dependence of the optics internal to the cryostat, not including the "on-wafer" bandpass function.  
 - $A\Omega \propto \nu^{-2}$ for a single-mode detector.
 
The power seen by a detector, coupled through the FTS, is
$$ P_{fts} = \int_0^\infty {\cal P}_{fts}(\nu) d\nu ,$$
where ${\cal P}_{fts}(\nu)$ is the power spectral density of the coupled radiation.  When we plot the raw FTS spectrum vs $\nu$, we are really plotting ${\cal P}_{fts}(\nu)$.
We can thus write:

$$ {\cal P}_{fts}(\nu) = B_\nu(\nu,T_{src}) \cdot A\Omega(\nu) \cdot F_{splitter}(\nu) \cdot F_{optics}(\nu) \cdot F_{detector} $$

The temperature of the "hot" load in an FTS is typically around ambient, putting our millimeter-wave detector bands well onto the Rayleigh-Jeans (RJ) side of the blackbody emission spectrum of that source.  In that case, we can use $B_\nu(\nu,T_{src}) \propto \nu^2$.

We can easily use the equation above to solve for the transmission of the "on-wafer" filters, for use in bolo-calc or for comparison with models of those filters.  We find

$$ F_{detector} = \frac{{\cal P}_{fts}(\nu)}{B_\nu(\nu,T_{src}) \cdot A\Omega(\nu) \cdot F_{splitter}(\nu) \cdot F_{optics}(\nu) } $$

$$ F_{detector} \propto \frac{{\cal P}_{fts}(\nu)}{\nu^2 \cdot \nu^{-2} \cdot \nu^2 \cdot F_{optics}(\nu)} $$

$$ F_{detector} \propto \frac{{\cal P}_{fts}(\nu)}{\nu^2 \cdot F_{optics}(\nu)} \tag{1} $$
where we recall that $F_{optics}(\nu)$ includes the effects of frequency-dependent loss in the cryostat optics, including spillover on the cold stop.

We're also interested in the function $F_{on-sky}$ used to calculate the coupling of external (outside the cryostat) sources to the detector;  for example, what is the frequency dependence of the detector's coupling to the atmosphere, the CMB or galactic dust?

Here, we write the spectral density of that coupling similar to the FTS coupling above, as

$$ {\cal P}_{sky}(\nu) = B_\nu(\nu,T_{src}) \cdot A\Omega(\nu) \cdot F_{optics}(\nu) \cdot F_{detector}$$

$$ {\cal P}_{sky}(\nu) = B_\nu(\nu,T_{src}) \cdot A\Omega(\nu) \cdot F_{on-sky}(\nu)$$

which makes clear that the on-sky filter function we desire is

$$ F_{on-sky}(\nu) = F_{optics}(\nu) \cdot F_{detector},$$

which in retrospect seems obvious.  Looking at the solution for $F_{detector}$ above, we see that we can solve for $F_{on-sky}(\nu)$ by moving $F_{detector}$ to the LHS, giving

$$ F_{on-sky}(\nu) \propto \frac{{\cal P}_{fts}(\nu)}{\nu^2} . \tag{2}$$

Equations (1) and (2) are the main results of this discussion.



In [6]:
# Standard imports
import numpy as np
import yaml
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  
plt.rcParams.update({'font.size': 18})
plt.rcParams['figure.figsize'] = [12, 20]

# define the blackbody function.
def Bnu_1pol(nu,T):  # Calculate the planck blackbody brightness in W/m^2/sr/Hz
    '''
    Planck blackbody brightness function (1 polarization, in W/m^2/sr/Hz).
    Frequency (nu) must be in Hz.
    Bnu(nu,T) = n_polarizations*(h*nu)*(nu/c)**2 * (1./(numpy.exp(x) - 1.))
    '''

    # Frequency vector must be in Hz.
    # Physical constants in SI units
    c= 2.99792458e8  
    h = 6.626068e-34
    k = 1.3806503e-23
    x = (h*nu)/(k*T)

    # Factor of 1 on RHS means this is for 1 polarization
    brightness = 1.*(h*nu)*(nu/c)**2 * (1./(numpy.exp(x) - 1.))

    return brightness